In [25]:
import pandas as pd
import numpy as np  

In [26]:
kurlamain = pd.read_csv('Data/Kurla_dataset.csv')
airportmain = pd.read_csv('Data/Airport_dataset.csv')
chakalamain = pd.read_csv('Data/Chakala_dataset.csv')

In [38]:
# display(kurlamain)
# display(airportmain)
# display(chakalamain)
timeStamp=kurlamain['From Date']


In [39]:
kurlamain.replace('None', np.nan, inplace=True)
kurlamain
airportmain.replace('None', np.nan, inplace=True)
airportmain
chakalamain.replace('None', np.nan, inplace=True)
chakalamain

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,Temp
0,17-03-2021 00:00,17-03-2021 01:00,170.50,359.65,240.83,109.15,15.46,2.25,4.25,43.45,0.10,253.77,26.19
1,17-03-2021 01:00,17-03-2021 02:00,124.88,268.00,6.14,54.65,15.75,1.00,9.50,66.59,0.10,70.40,26.15
2,17-03-2021 02:00,17-03-2021 03:00,89.41,194.18,6.85,42.96,15.29,1.00,13.50,72.55,0.08,71.60,25.66
3,17-03-2021 03:00,17-03-2021 04:00,89.13,194.54,5.27,44.89,13.88,1.00,13.25,66.35,0.09,257.74,25.41
4,17-03-2021 04:00,17-03-2021 05:00,97.21,211.69,11.30,64.55,10.70,1.00,7.00,60.13,0.09,257.77,24.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17532,17-03-2023 12:00,17-03-2023 13:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17533,17-03-2023 13:00,17-03-2023 14:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17534,17-03-2023 14:00,17-03-2023 15:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17535,17-03-2023 15:00,17-03-2023 16:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52


Processing the missing data with "bfill" & "ffill"

In [40]:
# kurlafilled=kurlamain.fillna(method='ffill', inplace=False)
# kurlafilled=kurlamain.fillna(method='bfill', inplace=False)
# display(kurlafilled)
# airportfilled=airportmain.fillna(method='ffill', inplace=False)
# airportfilled=airportmain.fillna(method='bfill', inplace=False)
# display(airportfilled)
# chakalafilled=chakalamain.fillna(method='ffill', inplace=False)
# chakalafilled=chakalamain.fillna(method='bfill', inplace=False)
# display(chakalafilled)

Processing the missing data with Interpolate

In [41]:
kurlainter = kurlamain
kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].interpolate(method ='linear', limit_direction ='both')
kurlainter

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
0,17-03-2021 00:00,17-03-2021 01:00,137.96,272.00,7.97,129.990000,73.270,2.320,3.470,44.770000,0.820000,236.970000,25.740000
1,17-03-2021 01:00,17-03-2021 02:00,111.31,272.00,7.97,94.086667,67.162,1.908,3.566,50.273333,0.846667,219.323333,25.523333
2,17-03-2021 02:00,17-03-2021 03:00,49.00,272.00,7.97,58.183333,61.054,1.496,3.662,55.776667,0.873333,201.676667,25.306667
3,17-03-2021 03:00,17-03-2021 04:00,32.83,272.00,7.97,22.280000,54.946,1.084,3.758,61.280000,0.900000,184.030000,25.090000
4,17-03-2021 04:00,17-03-2021 05:00,50.19,272.00,17.54,42.045000,48.838,0.672,3.854,60.695000,0.815000,151.750000,24.715000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17559,18-03-2023 15:00,18-03-2023 16:00,14.93,217.88,10.18,25.750000,25.550,0.330,49.060,62.990000,2.590000,231.420000,32.500000
17560,18-03-2023 16:00,18-03-2023 17:00,17.07,250.22,12.90,24.920000,25.540,0.260,44.690,66.230000,2.860000,242.130000,31.920000
17561,18-03-2023 17:00,18-03-2023 18:00,12.36,236.92,10.55,22.700000,25.560,0.230,48.020,70.650000,2.540000,243.530000,31.170000
17562,18-03-2023 18:00,18-03-2023 19:00,13.25,189.68,10.98,24.550000,25.550,0.250,43.670,77.600000,2.050000,244.630000,30.360000


In [42]:
airportinter = airportmain
airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].interpolate(method ='linear', limit_direction ='both')
airportinter

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
0,17-03-2021 00:00,17-03-2021 01:00,101.24,429.09,246.44,28.04,8.45,2.14,5.99,53.67,0.55,254.73,28.03
1,17-03-2021 01:00,17-03-2021 02:00,91.22,394.22,175.60,15.24,7.25,1.64,16.93,69.27,0.82,289.63,27.62
2,17-03-2021 02:00,17-03-2021 03:00,37.07,167.11,146.64,9.93,4.58,1.34,31.58,75.85,0.82,251.97,27.07
3,17-03-2021 03:00,17-03-2021 04:00,37.95,129.33,128.77,9.23,6.04,1.23,35.86,71.35,0.67,139.85,26.80
4,17-03-2021 04:00,17-03-2021 05:00,35.06,144.67,114.89,11.63,5.24,1.09,12.45,59.59,0.42,251.75,26.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17532,17-03-2023 12:00,17-03-2023 13:00,25.70,110.58,67.62,48.20,8.39,1.26,23.91,54.96,1.75,186.31,31.80
17533,17-03-2023 13:00,17-03-2023 14:00,21.88,107.47,63.57,49.73,9.09,1.24,23.71,58.73,1.01,273.52,32.49
17534,17-03-2023 14:00,17-03-2023 15:00,25.43,139.15,46.97,42.34,8.57,1.35,23.89,61.13,1.08,281.31,31.86
17535,17-03-2023 15:00,17-03-2023 16:00,26.56,100.60,44.68,39.01,8.59,1.20,23.70,61.51,0.92,271.54,32.06


In [43]:
chakalainter = chakalamain
chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']]= chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']].astype(float)
chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']]= chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']].interpolate(method ='linear', limit_direction ='both')
chakalainter 

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,Temp
0,17-03-2021 00:00,17-03-2021 01:00,170.50,359.65,240.83,109.15,15.46,2.25,4.25,43.45,0.10,253.77,26.19
1,17-03-2021 01:00,17-03-2021 02:00,124.88,268.00,6.14,54.65,15.75,1.00,9.50,66.59,0.10,70.40,26.15
2,17-03-2021 02:00,17-03-2021 03:00,89.41,194.18,6.85,42.96,15.29,1.00,13.50,72.55,0.08,71.60,25.66
3,17-03-2021 03:00,17-03-2021 04:00,89.13,194.54,5.27,44.89,13.88,1.00,13.25,66.35,0.09,257.74,25.41
4,17-03-2021 04:00,17-03-2021 05:00,97.21,211.69,11.30,64.55,10.70,1.00,7.00,60.13,0.09,257.77,24.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17532,17-03-2023 12:00,17-03-2023 13:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17533,17-03-2023 13:00,17-03-2023 14:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17534,17-03-2023 14:00,17-03-2023 15:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52
17535,17-03-2023 15:00,17-03-2023 16:00,181.72,255.29,66.47,210.11,28.82,3.67,68.33,27.90,0.09,314.53,32.52


Outliers using STD

In [44]:
kurlastd = kurlainter
kurlastd = kurlastd.drop(kurlastd.columns[[0,1]], axis=1)
kurlastd

,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
0,137.96,272.00,7.97,129.990000,73.270,2.320,3.470,44.770000,0.820000,236.970000,25.740000
1,111.31,272.00,7.97,94.086667,67.162,1.908,3.566,50.273333,0.846667,219.323333,25.523333
2,49.00,272.00,7.97,58.183333,61.054,1.496,3.662,55.776667,0.873333,201.676667,25.306667
3,32.83,272.00,7.97,22.280000,54.946,1.084,3.758,61.280000,0.900000,184.030000,25.090000
4,50.19,272.00,17.54,42.045000,48.838,0.672,3.854,60.695000,0.815000,151.750000,24.715000
...,...,...,...,...,...,...,...,...,...,...,...
17559,14.93,217.88,10.18,25.750000,25.550,0.330,49.060,62.990000,2.590000,231.420000,32.500000
17560,17.07,250.22,12.90,24.920000,25.540,0.260,44.690,66.230000,2.860000,242.130000,31.920000
17561,12.36,236.92,10.55,22.700000,25.560,0.230,48.020,70.650000,2.540000,243.530000,31.170000
17562,13.25,189.68,10.98,24.550000,25.550,0.250,43.670,77.600000,2.050000,244.630000,30.360000


In [45]:
def determine_outlier_thresholds_std(dataframe, col_name):
    upper_boundary = dataframe[col_name].mean() + 3 * dataframe[col_name].std()
    lower_boundary = dataframe[col_name].mean() - 3 * dataframe[col_name].std()
    return lower_boundary, upper_boundary
def check_outliers_std(dataframe, col_name):
    lower_boundary, upper_boundary = determine_outlier_thresholds_std(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_boundary) | (dataframe[col_name] < lower_boundary)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_std(dataframe, cols, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_std(dataframe, col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_std(dataframe, col_name)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                if replace:
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_std(dataframe, col_name)
            data.append([outliers_, outliers_status,count, col_name, lower_limit, upper_limit])
    table = tabulate(data, headers=['Outlier (Previously)','Outliers','Count', 'Column','Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    # print("Removing Outliers using 3 Standard Deviation")
    # print(table)

display(kurlastd.describe())
replace_with_thresholds_std(kurlastd, kurlastd.columns,replace=True)
display(kurlastd.describe())

,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
count,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000
mean,46.329440,160.263735,44.421007,36.648477,28.147517,0.531724,45.107873,73.444025,1.635350,187.514076,27.908467
std,55.638182,133.569794,44.357425,31.808254,30.287911,0.450135,33.295435,18.603246,0.987613,74.178668,4.990215
min,1.000000,0.610000,0.020000,0.010000,0.010000,0.000000,0.010000,17.040000,0.260000,27.130000,14.240000
25%,18.770000,61.927500,13.690000,11.823750,12.180000,0.235000,21.648051,60.780000,1.080000,108.951500,24.920000
50%,31.250000,131.995000,30.512544,28.315000,14.823421,0.424442,40.010000,76.225000,1.530000,215.065000,27.040256
75%,55.472500,225.712500,57.455000,51.791167,32.989603,0.670000,57.370000,87.742500,2.060000,252.370000,29.960000
max,812.620000,985.000000,391.470000,383.680000,199.870000,5.050000,191.680000,99.940000,43.980000,335.740000,48.520000


,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
count,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000,17564.000000
mean,43.644263,157.121882,43.468998,36.444457,27.161756,0.522210,44.923436,73.444059,1.614770,187.514076,27.867824
std,37.373414,120.124248,40.433410,31.032394,26.323818,0.409301,32.660586,18.603144,0.709993,74.178668,4.853162
min,1.000000,0.610000,0.020000,0.010000,0.010000,0.000000,0.010000,17.634286,0.260000,27.130000,14.240000
25%,18.770000,61.927500,13.690000,11.823750,12.180000,0.235000,21.648051,60.780000,1.080000,108.951500,24.920000
50%,31.250000,131.995000,30.512544,28.315000,14.823421,0.424442,40.010000,76.225000,1.530000,215.065000,27.040256
75%,55.472500,225.712500,57.455000,51.791167,32.989603,0.670000,57.370000,87.742500,2.060000,252.370000,29.960000
max,213.243986,560.973118,177.493283,132.073240,119.011250,1.882128,144.994178,99.940000,4.598187,335.740000,42.879113


Outliers using IQR

In [46]:
kurlaiqr = kurlainter
kurlaiqr = kurlaiqr.drop(kurlaiqr.columns[[0,1]], axis=1)
# kurlaiqr = kurlaiqr.apply(pd.to_numeric, errors='coerce')


In [ ]:
def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.25, th3=0.75):
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit

def check_outliers_iqr(dataframe, col_name):
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_iqr(dataframe,cols, th1=0.05, th3=0.95, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                if replace: 
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(dataframe, col_name)
            data.append([outliers_, outliers_status, count,  col_name, lower_limit, upper_limit ])
    table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count' ,'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    # print("Removing Outliers using IQR")
    # print(table)
    
display(kurlaiqr.describe())
replace_with_thresholds_iqr(kurlaiqr, kurlaiqr.columns, replace=True)
display(kurlaiqr.describe())


In [50]:
kurlastd['Date']=timeStamp
kurlastd.to_csv('kurla_cleaned.csv')